In [1]:
# %load imports.py
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import scipy as sp
import scipy.ndimage as snd
import scipy.signal as ss
import skimage.morphology as skm
import mkl_fft
import fast_histogram
import sys, os, datetime, shutil, glob, itertools, collections, pathlib
from pathlib import Path
import h5py
from tqdm.notebook import tqdm
from pprint import pprint
from sacla import *
from accum import *
import idi.reconstruction as recon

%matplotlib inline
pd.options.display.max_rows = 999
%reload_ext line_profiler
%reload_ext memory_profiler


def browse(obj, depth=0):
    try:
        if type(obj) == h5py._hl.dataset.Dataset:
            print(obj.shape, obj.dtype, end='')
            if np.product(obj.shape) < 10:
                print(np.array(obj), end='')
        else:
            for k in obj.keys():
                print('\n', depth * '   ', k, end='')
                browse(obj[k], depth + 1)
            print()
    except Exception as e:
        print(e)


def zoom(cor):
    cor[cor.shape[0] // 2, cor.shape[1] // 2] = np.nan
    plt.matshow(cor[cor.shape[0] // 2 - n : cor.shape[0] // 2 + n, cor.shape[1] // 2 - n : cor.shape[1] // 2 + n])

/home/gorkhover/anaconda3/envs/zimmf4/lib/python3.8/site-packages/idi/reconstruction/__init__.py:60: UserWarning: no cuda available
  _w.warn("no cuda available")


In [2]:
l=log('sacla2020.csv')

In [3]:
l[0:110]

,Run start,last run,Name,Taget,Target Material,Pre-Run Comment,Post-Run Comment,Filter Oktal,Filter Dual,Distance_Dual,Distance_Oktal,focus y,p1,p2_stats,p2_thres1000,p2_thres1500,p2_thres2000,Att,Defocus
,#,#,Name,ID,Name,Text,Text,Material,Material,cm,cm,um,Path,Path,Path,Path,Path,mm Silicon,um
0,918261,918261,dark,NaN,NaN,dark run,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,918280,918289,background,2,None,Background recording. Motor is still scanning.,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,918308,918348,Cu20um_focus,3,Cu20um,NaN,On the dual there are photons with ~160ADU and...,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,918349,918368,Cu20um_defocus-500,3,Cu20um,NaN,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,918369,918413,Fe500nm_focus,4,Fe500nm,Try 0.5um iron foil. get back into focus. Also...,"Diffraction visible, see SLACK. Live analysis ...",None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,918414,918456,Fe500nm_defocus-200,4,Fe500nm,moved out of focus (-200um),foil entirely shot\r\n,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,918457,918467,dark,none,none,dark runs for detector. no FEL,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,918468,918510,Fe10um_focus,6,Fe10um,NaN,NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,918511,918551,Fe10um_defocus-400,6,Fe10um,moved out of focus (-400um),NaN,None,None,100.0,43.0,-210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Foils

In [4]:
start=2
end=20
basepath=Path('/arcv1/UserData/gorkhover/TAIS2020/foils/')

In [22]:
##check on inputfiles

runs=collections.defaultdict(list)
for _,s in l[start:end].iterrows():
    name=str(s['Name'][0])
    if '?' in name: continue
    if 'discard' in name: continue
    for i in range(int(s['Run start']),1+int(s['last run'])):
        try:
            t=h5py.File(basepath/f'{i}.h5','r')
            if not 'detector_2d_assembled_2' in list(t[list(t.keys())[1]].keys()):
                print('Oassembled',i)
            if not 'detector_2d_assembled_1' in list(t[list(t.keys())[1]].keys()):
                print('Dassembled',i)
        except Exception as e:
            print(i,e)
            pass

In [9]:
darkpath=basepath/'bg'
import glob
def get_dark(num,darkpath):
    darks={int(os.path.splitext(os.path.basename(f))[0].split('_')[1]):f for f in sorted(glob.glob(os.path.join(darkpath,'*.h5')))}
    return darks.get(num, darks[min(darks.keys(), key=lambda k: abs(k-num) if k<=num else 1000)])

import collections
runs=collections.defaultdict(list)
focus={}
for _,s in l[start:end].iterrows():

    name=str(s['Name'][0])
    focus[name]=float(s['focus y'])*1e-6
    if '?' in name: continue
    if 'dark' in name: continue
    if 'discard' in name: continue
    if 'background' in name: continue
    for i in range(int(s['Run start']),1+int(s['last run'])):
        if len(runs[name])>=40: break
        runs[name].append(i)
    print(name,len(runs[name]),runs[name][0],runs[name][-1],get_dark(runs[name][0],darkpath),focus[name])

Cu20um_focus 40 918308 918347 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Cu20um_defocus-500 20 918349 918368 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Fe500nm_focus 40 918369 918408 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Fe500nm_defocus-200 40 918414 918453 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Fe10um_focus 40 918468 918507 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Fe10um_defocus-400 40 918511 918550 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Fe500nm_focus_2 40 918552 918591 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Fe500nm_defocus+400 40 918594 918633 /arcv1/UserData/gorkhover/TAIS2020/foils/bg/bg_918457_918467.h5 -0.00020999999999999998
Cu5um_focus 40 918647 9186

In [10]:
inputpath=str(basepath/"{number}.h5")
#bgfile="/work/gorkhover/TAIS2020/{name}.h5"
outputpath=basepath/'p1'/'final'
outputpath.mkdir(parents=True, exist_ok=True)
outputpath=str(outputpath/"{name}_p1.h5")

cmds=[]
for name,r in runs.items():
    #if not (name=='Fe500nm_focus' or name=='Cu5um_focus'):continue
    #if not any([k in name for k in ['20nmCo_+400']]): continue
    dark=get_dark(r[0],darkpath)
    inputfiles=' '.join([inputpath.format(number=str(i)) for i in r])
    outputfile=outputpath.format(name=name)
    sample=name.split("_")[0]
    focus_y=focus[name]
    if 'Cu' == name[:2]:
        kalpha=8046
    elif 'Ni' == name[3:5]:
        kalpha=7480
    elif 'Fe' == name[:2] or any(e in name.lower() for e in ['nmco','nmps','nmspin','nmpmma','copolymeronly']):
        kalpha=6405
    else:
        print(f'error: dont know material of {name}')
        continue
    cmd=f"python ~/zimmf/msc/sacla/phase1.py {inputfiles} {outputfile} --bgfile {dark} --name {name} --kalpha {kalpha} --sample {sample} --focus_y {focus_y:.6f}"
    cmds.append((name,cmd))

In [11]:
for name,cmd in cmds:
        #if not (name == 'Fe500nm_focus'): 
        #    print('skipped',name)
        #    continue
        print(cmd)
        print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf4',cmd],jobname=f"p1f_{name}",threads=14))

python ~/zimmf/msc/sacla/phase1.py /arcv1/UserData/gorkhover/TAIS2020/foils/918308.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918309.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918310.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918311.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918312.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918313.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918314.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918315.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918316.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918317.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918318.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918319.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918320.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918321.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918322.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918323.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918324.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918325.h5 /arcv1/UserData/gorkhover/TAIS2020/foils/918326

## nano ##

In [12]:
start=18
end=100
basepath=Path('/arcv1/UserData/gorkhover/TAIS2020/nano/')

In [13]:
##check on inputfiles

runs=collections.defaultdict(list)
for _,s in l[start:end].iterrows():
    name=str(s['Name'][0])
    if '?' in name: continue
    if 'discard' in name: continue
    for i in range(int(s['Run start']),1+int(s['last run'])):
        try:
            t=h5py.File(basepath/f'{i}.h5','r')
            if not 'detector_2d_assembled_2' in list(t[list(t.keys())[1]].keys()):
                print('Oassembled',i)
            if not 'detector_2d_assembled_1' in list(t[list(t.keys())[1]].keys()):
                print('Dassembled',i)
        except Exception as e:
            print(i,e)
            pass

In [18]:
darkpath=basepath/'bg'
import glob
def get_dark(num,darkpath):
    darks={int(os.path.splitext(os.path.basename(f))[0].split('_')[1]):f for f in sorted(glob.glob(os.path.join(darkpath,'*.h5')))}
    return darks.get(num, darks[min(darks.keys(), key=lambda k: abs(k-num) if k<=num else 1000)])

import collections
runs=collections.defaultdict(list)
focus={}
for _,s in l[start:end].iterrows():

    name=str(s['Name'][0])
    focus[name]=float(s['focus y'])*1e-6
    if '?' in name: continue
    if 'dark' in name: continue
    if 'discard' in name: continue
    if 'background' in name: continue
    for i in range(int(s['Run start']),1+int(s['last run'])):
        if len(runs[name])>=45: 
            print(name, 'has more than 45, skipping rest')
        runs[name].append(i)
    print(name,len(runs[name]),runs[name][0],runs[name][-1],get_dark(runs[name][0],darkpath),focus[name])

10nmPS_focus_far_nofilter 20 918875 918894 /arcv1/UserData/gorkhover/TAIS2020/nano/bg/bg_918639_918665.h5 -0.00015999999999999999
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter has more than 45, skipping rest
10nmPS_focus_far_nofilter 53 918875 918927 /arcv1/UserData/gorkhover/TAIS2020/nano/bg/bg_918639_918665.h5 -0.00015999999999999999
10nmPS_-400_att_nofilter 11 918930 918940 /arcv1/UserData/gorkhover/TAIS2020/nano/bg/bg_918639_918665.h5 -0.00015999999999999999
10nmPS_-400_full_nofilter 29 918941 918969 /arcv1/UserData/gorkhover/TAIS2020/nano/bg/bg_918639_918665.h5 -0.00015999999999999999
10nmPS_focus_nofi

In [22]:
inputpath=str(basepath/"{number}.h5")
#bgfile="/work/gorkhover/TAIS2020/{name}.h5"
outputpath=basepath/'p1'/'final'
outputpath.mkdir(parents=True, exist_ok=True)
outputpath=str(outputpath/"{name}_p1.h5")

cmds=[]
for name,r in runs.items():
    #if not (name=='Fe500nm_focus' or name=='Cu5um_focus'):continue
    #if not any([k in name for k in ['20nmCo_+400']]): continue
    dark=get_dark(r[0],darkpath)
    inputfiles=' '.join([inputpath.format(number=str(i)) for i in r])
    outputfile=outputpath.format(name=name)
    sample=name.split("_")[0]
    focus_y=focus[name]
    if 'Cu' == name[:2]:
        kalpha=8046
    elif 'Ni' == name[3:5]:
        kalpha=7480
    elif 'Fe' == name[:2] or any(e in name.lower() for e in ['nmco','nmps','nmspin','nmpmma','copolymeronly']):
        kalpha=6405
    else:
        print(f'error: dont know material of {name}')
        continue
    cmd=f"python ~/zimmf/msc/sacla/phase1.py {inputfiles} {outputfile} --bgfile {dark} --name {name} --kalpha {kalpha} --sample {sample} --focus_y {focus_y:.6f}"
    cmds.append((name,cmd))

In [23]:
for name,cmd in cmds:
#         if not (name == '10nmPS_focus_far_nofilter'): 
#             print('skipped',name)
#             continue
        print(cmd)
        print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf4',cmd],jobname=f"p1n_{name}",threads=14))

python ~/zimmf/msc/sacla/phase1.py /arcv1/UserData/gorkhover/TAIS2020/nano/918875.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918876.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918877.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918878.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918879.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918880.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918881.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918882.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918883.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918884.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918885.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918886.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918887.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918888.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918889.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918890.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918891.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918892.h5 /arcv1/UserData/gorkhover/TAIS2020/nano/918893.h5 /arcv1/UserData

## crystal ##

In [40]:
start=105
end=112
basepath=Path('/arcv1/UserData/gorkhover/TAIS2020/crystal/')

In [41]:
darkpath=basepath/'bg'
import glob
def get_dark(num,darkpath):
    darks={int(os.path.splitext(os.path.basename(f))[0].split('_')[1]):f for f in sorted(glob.glob(os.path.join(darkpath,'*.h5')))}
    return darks.get(num, darks[min(darks.keys(), key=lambda k: abs(k-num) if k<=num else 1000)])

import collections
runs=collections.defaultdict(list)
focus={}
for _,s in l[start:end].iterrows():

    name=str(s['Name'][0])
    focus[name]=float(s['focus y'])*1e-6
    if '?' in name: continue
    if 'dark' in name: continue
    if 'discard' in name: continue
    if 'background' in name: continue
    for i in range(int(s['Run start']),1+int(s['last run'])):
        if len(runs[name])>=45: 
            print(name, 'has more than 45, skipping rest')
        runs[name].append(i)
    print(name,len(runs[name]),runs[name][0],runs[name][-1],get_dark(runs[name][0],darkpath),focus[name])

gaas1 15 919744 919758 /arcv1/UserData/gorkhover/TAIS2020/crystal/bg/bg_919786.h5 4.9999999999999996e-06
gaas2 21 919764 919784 /arcv1/UserData/gorkhover/TAIS2020/crystal/bg/bg_919786.h5 4.9999999999999996e-06


In [43]:
inputpath=str(basepath/"{number}.h5")
outputpath=basepath/'p1'/'final'
outputpath.mkdir(parents=True, exist_ok=True)
outputpath=str(outputpath/"{name}_p1.h5")

cmds=[]
for name,r in runs.items():
    #if not (name=='Fe500nm_focus' or name=='Cu5um_focus'):continue
    #if not any([k in name for k in ['20nmCo_+400']]): continue
    dark=get_dark(r[0],darkpath)
    inputfiles=' '.join([inputpath.format(number=str(i)) for i in r])
    outputfile=outputpath.format(name=name)
    sample=name.split("_")[0]
    focus_y=focus[name]
    if 'Cu' == name[:2]:
        kalpha=8046
    elif 'Ni' == name[3:5]:
        kalpha=7480
    elif 'Fe' == name[:2] or any(e in name.lower() for e in ['nmco','nmps','nmspin','nmpmma','copolymeronly']):
        kalpha=6405
    elif 'gaas' in name:
        kalpha=9200
    else:
        print(f'error: dont know material of {name}')
        continue
    #MKL_THREADING_LAYER=GNU 
    cmd=f"python ~/zimmf/msc/sacla/phase1.py {inputfiles} {outputfile} --pulsethres 300 --bgfile {dark} --name {name} --kalpha {kalpha} --sample {sample} --focus_y {focus_y:.6f}"
    cmds.append((name,cmd))

In [44]:
for name,cmd in cmds:
        #if not (name =='Cu5um_focus' or 'Fe500nm_focus_high' in name): continue
        print(cmd)
        print(qsub(['eval "$(conda shell.bash hook)"','conda activate zimmf4',cmd],jobname=f"p1c_{name}",threads=14))

python ~/zimmf/msc/sacla/phase1.py /arcv1/UserData/gorkhover/TAIS2020/crystal/919744.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919745.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919746.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919747.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919748.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919749.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919750.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919751.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919752.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919753.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919754.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919755.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919756.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919757.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/919758.h5 /arcv1/UserData/gorkhover/TAIS2020/crystal/p1/final/gaas1_p1.h5 --pulsethres 300 --bgfile /arcv1/UserData/gorkhover/TAIS2020/crystal/bg/bg_919786.h5 --name gaas1 --kalpha